# Exercise 1: What can LLMs do and can't do

In [11]:
# set your open ai key in .env file (text file) as OPENAI_API_KEY=your-key
# your .env file should be in tthe same directory as this file
from dotenv import load_dotenv
_ = load_dotenv()

In [16]:
# pip install langchain langchain-openai

from langchain_openai import ChatOpenAI

# Create a simple chat model
llm = ChatOpenAI(model="gpt-4o-mini")  # or another chat model

# 1) LLMs are great: ask for some public facts → they tell you.
public_facts_prompt = (
    "List three well-known public facts about Florida in bullet points."
)
public_facts_response = llm.invoke(public_facts_prompt)
print("=== Public facts example ===")
print(public_facts_response.content)
print()

# 2) Write a customer email → done.
customer_email_prompt = (
    "Write a short, polite email to a customer explaining that their order "
    "shipment is delayed by two days, and reassure them."
)
customer_email_response = llm.invoke(customer_email_prompt)
print("=== Customer email example ===")
print(customer_email_response.content)

=== Public facts example ===
- **Theme Parks**: Florida is home to several famous theme parks, including Walt Disney World Resort in Orlando, Universal Studios, and SeaWorld, making it a major tourist destination.

- **Beaches**: The state boasts over 1,300 miles of coastline and is known for its beautiful beaches, such as Miami Beach, Clearwater Beach, and the Florida Keys, attracting sun-seekers from around the world.

- **Hurricanes**: Florida is located in a region prone to hurricanes and tropical storms, particularly during the Atlantic hurricane season, which runs from June 1 to November 30 each year.

=== Customer email example ===
Subject: Update on Your Order Shipment

Dear [Customer's Name],

I hope this message finds you well. I’m reaching out to inform you that there has been a slight delay in the shipment of your order. It is now expected to ship in two days.

We understand how important it is for you to receive your items on time, and we sincerely apologize for any inconv

## Illustrate what LLMs can't do by themselves

In [24]:
# pip install langchain langchain-openai
# Make sure OPENAI_API_KEY is set in your environment.

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")  # any OpenAI chat model

# 1) Ask as if it can check Florida weather and book tickets
weather_and_booking_prompt = """
You are connected to live weather data and a travel booking system.

Task:
1) Check the weather forecast for Miami, Florida for next week.
2) Choose the best 3-day window with good weather.
3) Book round-trip flight tickets from New York to Miami for those dates.
4) Confirm the booking details (dates, airline, times, and price).

If you cannot actually do these steps, explain clearly what you *can* do instead.
"""

weather_response = llm.invoke(weather_and_booking_prompt)
print("=== Florida weather and booking attempt ===")
print(weather_response.content)
print()

# 2) Ask as if it can read past transactions and customer history
customer_history_prompt = """
You have direct access to our live customer database and transaction history.

Customer ID: 12345

Task:
1) Read this customer's past 12 months of purchases and support tickets.
2) Summarize their purchase and support history.
3) Then write a personalized, meaningful email that references specific past orders
   and issues to thank them and offer a relevant promotion.

If you cannot actually read real data, explain clearly what you *can* do instead.
"""

customer_response = llm.invoke(customer_history_prompt)
print("=== Customer history and meaningful email attempt ===")
print(customer_response.content)

=== Florida weather and booking attempt ===
I'm unable to access live weather data or a travel booking system to perform the tasks you've requested. However, I can guide you on how to do it:

### Step-by-Step Process

1. **Check the Weather Forecast:**
   - Use a reliable weather website or app (like Weather.com, AccuWeather, or your local meteorological service).
   - Search for Miami, Florida, and check the forecast for the next week.
   - Look for a 3-day window with clear skies, mild temperatures, and no significant precipitation.

2. **Choose the Best 3-Day Window:**
   - After checking the forecast, determine which days have the best overall weather conditions.

3. **Book Flight Tickets:**
   - Visit a travel booking website (such as Expedia, Google Flights, or a specific airline's website).
   - Input the round-trip details: departure city (New York), destination (Miami), and the preferred travel dates based on your weather checks.
   - Compare flight options based on airlines, 